# Instituto de Educação Superior de Brası́lia – IESB
## Pós-Graduação em Inteligência Artificial
### Disciplina de Computação Cognitiva 3 / Turma 2021-1
#### Trabalho Final - Análise de Sentimento

                          EQUIPE:
                          - LUCAS DE SOUSA BRITO, MAT:2186330019, TURMA: 2021-1
                          - PABLO NOGUEIRA OLIVEIRA, MAT:2186330027, TURMA: 2021-1
                          - MATHEUS BARBOSA OLIVEIRA, MAT:2186330037, TURMA: 2021-1

# Análise de sentimento da base do twitter Sentiment140

### Dados de Origem

* http://help.sentiment140.com/for-students

| sentiment  | id | date | query_string | user | text
| ---        | -- | -    | -            | -    | ---
| 0=negativo | -  | -    | -            | -    | the original twitter message
| 2=neutro   | -  | -    | -            | -    | 
| 4=positivo | -  | -    | -            | -    |


```



In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

tf.random.set_seed(129783)
np.random.seed(3213)

# Importação

Dados de origem: 
* http://help.sentiment140.com/for-students
* colunas:
   * sentiment (0=negativo, 2=neutro, 4=positivo)
   * id
   * date
   * query_string
   * user
   * text

Para este exercício:
* apenas as colunas sentiment e text serão mantidas
* sentimentos neutros serão descartados
* sentimentos serão padronizados como 0=negativo e 1=positivo


In [3]:
def bloco():
    
    global df_original
    
    df_cols = ['sentiment','id','date','query_string','user','text']

    df_original = pd.read_csv(
        "training.1600000.processed.noemoticon.csv",
        header=None, 
        names=df_cols,
        encoding = "ISO-8859-1"
    )

    df_original.drop(
        ['id','date','query_string','user'],
        axis=1,
        inplace=True
    )
    df_original = df_original[ df_original['sentiment'] != 2 ] 
    df_original['sentiment'] = df_original['sentiment'].apply( lambda x: 1 if x==0 else 0 )
    return df_original

bloco()

,sentiment,text
0,1,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,is upset that he can't update his Facebook by ...
2,1,@Kenichan I dived many times for the ball. Man...
3,1,my whole body feels itchy and like its on fire
4,1,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,0,Just woke up. Having no school is the best fee...
1599996,0,TheWDB.com - Very cool to hear old Walt interv...
1599997,0,Are you ready for your MoJo Makeover? Ask me f...
1599998,0,Happy 38th Birthday to my boo of alll time!!! ...


# Padronização (1)

* Parte 1
  * remove todas as tags
  * remove urls
  * remove identificadores de usuários 
  * remove caracteres unicode inválidos
  * remove carcteres não textuais
  * transforma tudo para minúsculas
* Parte 2
  * tokeniza usando o keras
* Parte 3
  * separa base de treinamento e de teste

In [4]:
#data_limit = 200000
max_words = 1000
max_len = 200

def bloco():
    
    global df_original
    global df_train
    global df_test   
   
    # PARTE 1 - Limpa o texto 
    import re
    pat1 = r'@[A-Za-z0-9]+'
    pat2 = r'https?://[A-Za-z0-9./]+'
    pat3 = r'<.*?>'
    pat4 = r'&.*?;'
    pat = r'|'.join((pat1, pat2, pat3, pat4))
    def tweet_cleaner(text):       
        text = re.sub(pat,'',text)
        try:
            text = text.decode("utf-8-sig").replace(u"\ufffd", "?")
        except:
            text = text
        text = re.sub("[^a-zA-Z]", " ", text)
        text = text.lower()
        return text

    df_original['text2'] = df_original['text'].apply( tweet_cleaner )

    
    # PARTE 2 - Tokeniza usando o Keras
    #tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words,lower=True, split=" ")
    #tokenizer.fit_on_texts(df_original['text2'])
    #df_original['text3'] = tokenizer.texts_to_sequences(df_original['text2'])
    #df_original['text4'] = tf.keras.preprocessing.sequence.pad_sequences(
    #    df_original['text3'], 
    #    maxlen=max_len,
    #    #padding='post',
    #    #truncating='post'        
    #).tolist()              
    
    global vectorizer
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(max_features=200, min_df=5, max_df=0.7)
    vectorizer.fit(df_original['text2']) 

    global df_example
    df_example = df_original.sample(15)
    df_example['text3'] = [ list(i) for i in vectorizer.transform( df_example['text2'] ).toarray() ]
    df_example
    
    # PARTE 3 - Separa amostra de treinamento e de teste
   
    # 1 = train | 0 = test
    df_original['rand'] = pd.Series([0,1]).sample(len(df_original), replace=True).array
    
    df_train_sz = 20000
    df_train = df_original[ df_original[ 'rand' ] == 1 ]
    df_train_positive = df_train[ df_train['sentiment'] == 0 ].sample(n=int(df_train_sz/2), replace=True)
    df_train_negative = df_train[ df_train['sentiment'] == 1 ].sample(n=int(df_train_sz/2), replace=True)
    df_train = pd.concat( [ df_train_negative, df_train_positive ] )    
    df_train = df_train.sample(frac=1.0).reset_index(drop=True)
        
    df_test_sz = 40000
    df_test = df_original[ df_original[ 'rand' ] == 0 ].sample(int(df_test_sz/2), replace=True)
    df_test = df_test.sample(frac=1.0).reset_index(drop=True)    
    
bloco()
df_example

,sentiment,text,text2,text3
562641,1,Eyes wide shzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz...,eyes wide shzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1596542,0,@SarY_ChaN,chan,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
557121,1,"Well, the question as to whether I should get ...",well the question as to whether i should get ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
1176192,0,Picnic in the sun,picnic in the sun,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1400171,0,Good morning! i'm outside and its a beautiful ...,good morning i m outside and its a beautiful ...,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ..."
1092245,0,@smoochyfroggy You wellcome! Have a nice Day!,you wellcome have a nice day,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
213310,1,Why is it so impossible to get an egg cream he...,why is it so impossible to get an egg cream he...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1026205,0,lady at olive garden garden ask me if i cared ...,lady at olive garden garden ask me if i cared ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
842287,0,Looking for domains. Great deals only.,looking for domains great deals only,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
292903,1,not wasting my money on a piercing that has a ...,not wasting my money on a piercing that has a ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Na tabela acima é possível ver os estagios da limpeza:
* **text** contém o texto original da base 
* **text2** contém o texto após limpeza de tags, urls, nomes de usuário, números e minusculas
* **text3** contém o texto codificado matriz de contadores


# Modelos - Clássicos

In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# Aplicando o Bagging com Arvores de Decisão
bag_model = BaggingRegressor(
    base_estimator=DecisionTreeRegressor(),
    n_estimators=100,
    max_samples=0.7,
    max_features=1.0,
    bootstrap=True,
    bootstrap_features=False,
    random_state=123,
    n_jobs=1,
    verbose=True,
)

lr_model = LogisticRegression(random_state=0)
nb_model = MultinomialNB()



In [6]:
X_train = vectorizer.transform(df_train['text2'])
y_train = df_train['sentiment'].values

print( f'X.shape={X_train.shape} y.shape={y_train.shape}' )

X.shape=(20000, 200) y.shape=(20000,)


In [7]:
bag_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)
nb_model.fit(X_train, y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished


MultinomialNB()

In [8]:
def block(y_test, y_pred, title):
    
    global confusion_mtx
    global confusion_mtx_pc
    
    confusion_mtx = tf.math.confusion_matrix( y_test, y_pred )

    confusion_mtx = pd.DataFrame( confusion_mtx.numpy() )
    confusion_mtx.loc['Total'] = confusion_mtx.sum(numeric_only=True) 
    confusion_mtx['total'] = confusion_mtx[0] + confusion_mtx[1]
    
    confusion_mtx_pc = confusion_mtx / len(y_test)
    
    fp = confusion_mtx.iloc[0,1] 
    fn = confusion_mtx.iloc[1,0]
    tn = confusion_mtx.iloc[0,0]
    tp = confusion_mtx.iloc[1,1] 
    
    total = (tp+tn+fp+fn)
    
    acc       = (tp+tn)/(tp+tn+fp+fn)
    recall    = tp/(tp+fn)
    f1        = (2*acc*recall)/(acc+recall)
    
    fdr  = fp/(fp+tp)
    fnr  = fn/(fn+tp)
    
    tpr = tp/(fn+tp)
    ppv  = tp/(fp+tp)
    
    from IPython.display import display, HTML
    display(HTML(f"""
    
        <style>
           .luc_confusion_mtx td {{ background: white!IMPORTANT; border: 0pt !IMPORTANT; text-align: center!IMPORTANT }}           
           td.luc_confusion_mtx_dp {{ width: 90pt; height: 90pt; background: #c0ffc0!IMPORTANT; border: 1pt solid black!IMPORTANT }} 
           td.luc_confusion_mtx_dn {{ width: 90pt; height: 90pt; background: #ffc0c0!IMPORTANT; border: 1pt solid black!IMPORTANT }}            
        </style>
        
        <h4>{title}</h4>
        <table class='luc_confusion_mtx'>
        <tr>
            <td></td>
            <td></td>
            <td colspan=2>Previsão</td>
            <td></td>
            <td rowspan=5 style='text-align: left!IMPORTANT'>
                    Acurácia<br><big><big>(TP+TN)/(total)</big></big> = {tp+tn}/{total} = <big>{(tp+tn)*100.0/total:2.1f}%</big><br>
                    <br><br>
                    Considerando que as duas classes (0=sentimento negativo;1=sentimento positivo) tem igual 
                    valor para esta análise, é importante maximizar a diagonal verde / minimizar a diagonal vermelha,
                    portanto os indicadores de Acurácia OU F1 são os mais indicados.<br>
                    <br>
                    Os indicadores de precisão e sensibilidade(recall) podem ser usados em conjunto, mas não são
                    muito intuitivos para este conjunto de dados pois mensuram da perspectiva do "sentimento positivo".
                    Em outras palavras, a sensibilidade indica quantos "sentimentos positivos" corretos foram encontrados
                    e a "precisão" indica do total apontado pelo modelo como "sentimento positivo", quantos eram. 
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>Negativo</td>
            <td>Positivo</td>
            <td></td>            
        </tr>
        <tr>
            <td rowspan=2>Real</td>
            <td>Negativo</td>
            <td class="luc_confusion_mtx_dp"><big><big>TN</big></big><br>{tn}<br>{tn*100.0/total:2.1f}%</td>
            <td class="luc_confusion_mtx_dn"><big><big>FP</big></big><br>{fp}<br>{fp*100.0/total:2.1f}%</td>
            <td>{tn+fp}</td>
        </tr>        
        <tr>
            <td>Positivo</td>
            <td class="luc_confusion_mtx_dn"><big><big>FN</big></big><br>{fn}<br>{fn*100.0/total:2.1f}%</td>
            <td class="luc_confusion_mtx_dp"><big><big>TP</big></big><br>{tp}<br>{tp*100.0/total:2.1f}%</td>
            <td>{fn+fp}</td>
        </tr>  
        <tr>
            <td></td>
            <td></td>
            <td>{tn+fn}</td>
            <td>{fp+tp}</td>
            <td>{total}</td>
        </tr>  
        </table>    
        
        
       
    """))        
    
    print( f'             Acurácia={acc*100.0:05.2f}% dos apontamentos positivos e negativos estão corretos' )
    print( f' Recall/Sensibilidade={tpr*100.0:05.2f}% dos sentimentos positivos da base foram apontados' )   
    print( f'             Precisão={ppv*100.0:05.2f}% dos sentimentos positivos apontados estão corretos' )    
    print( f'             F1 Score={f1*100.0:05.2f}% média harmônica da acurácia e recall' )        
        


In [9]:
X_test = vectorizer.transform( df_test['text2'] )
y_test = df_test['sentiment'].values
y_pred_bag = [ 1 if y_pred > 0.5 else 0 for y_pred in bag_model.predict(X_test) ]
y_pred_lr = [ 1 if y_pred > 0.5 else 0 for y_pred in lr_model.predict(X_test) ]
y_pred_nb = [ 1 if y_pred > 0.5 else 0 for y_pred in nb_model.predict(X_test) ]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s finished


In [10]:
block(y_test, y_pred_bag, 'Método Bagging com Árvore de Decisão')
block(y_test, y_pred_lr, 'Método Regressão Logística')
block(y_test, y_pred_nb, 'Método MultinomialNB')

2022-05-26 14:15:40.241031: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-26 14:15:40.241123: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: lucas-n001
2022-05-26 14:15:40.241138: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: lucas-n001
2022-05-26 14:15:40.241322: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.73.5
2022-05-26 14:15:40.241367: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.73.5
2022-05-26 14:15:40.241378: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.73.5
2022-05-26 14:15:40.243822: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions

             Acurácia=67.38% dos apontamentos positivos e negativos estão corretos
 Recall/Sensibilidade=65.22% dos sentimentos positivos da base foram apontados
             Precisão=68.09% dos sentimentos positivos apontados estão corretos
             F1 Score=66.28% média harmônica da acurácia e recall


             Acurácia=70.02% dos apontamentos positivos e negativos estão corretos
 Recall/Sensibilidade=65.95% dos sentimentos positivos da base foram apontados
             Precisão=71.70% dos sentimentos positivos apontados estão corretos
             F1 Score=67.92% média harmônica da acurácia e recall


             Acurácia=69.36% dos apontamentos positivos e negativos estão corretos
 Recall/Sensibilidade=69.29% dos sentimentos positivos da base foram apontados
             Precisão=69.32% dos sentimentos positivos apontados estão corretos
             F1 Score=69.33% média harmônica da acurácia e recall
